In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinovation-grades-2022/sample_submission.csv
/kaggle/input/tinovation-grades-2022/student_info.txt
/kaggle/input/tinovation-grades-2022/train.csv
/kaggle/input/tinovation-grades-2022/test.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/tinovation-grades-2022/train.csv",index_col=[0])
train_data['G4'] = (train_data['G1']+train_data['G2'])/2
test_data = pd.read_csv("/kaggle/input/tinovation-grades-2022/test.csv",index_col=[0])
# test_data['Id'] = list(range(0, 75))
test_data['G4'] = (test_data['G1']+test_data['G2'])/2
# print(train_data)

In [3]:
train_data['sex'] = train_data['sex'].eq('M').mul(1)
train_data['address'] = train_data['address'].eq('U').mul(1)
train_data['guardian'] = train_data['guardian'].eq('mother').mul(1)
train_data['schoolsup'] = train_data['schoolsup'].eq('yes').mul(1)
train_data['famsup'] = train_data['famsup'].eq('yes').mul(1)
train_data['paid'] = train_data['paid'].eq('yes').mul(1)
train_data['activities'] = train_data['activities'].eq('yes').mul(1)
train_data['nursery'] = train_data['nursery'].eq('yes').mul(1)
train_data['internet'] = train_data['internet'].eq('yes').mul(1)
train_data['romantic'] = train_data['romantic'].eq('yes').mul(1)

test_data['sex'] = test_data['sex'].eq('M').mul(1)
test_data['address'] = test_data['address'].eq('U').mul(1)
test_data['guardian'] = test_data['guardian'].eq('mother').mul(1)
test_data['schoolsup'] = test_data['schoolsup'].eq('yes').mul(1)
test_data['famsup'] = test_data['famsup'].eq('yes').mul(1)
test_data['paid'] = test_data['paid'].eq('yes').mul(1)
test_data['activities'] = test_data['activities'].eq('yes').mul(1)
test_data['nursery'] = test_data['nursery'].eq('yes').mul(1)
test_data['internet'] = test_data['internet'].eq('yes').mul(1)
test_data['romantic'] = test_data['romantic'].eq('yes').mul(1)
print(test_data)

    sex  age  address  guardian  studytime  failures  schoolsup  famsup  paid  \
0     0   18        1         0          1         3          1       0     1   
1     0   18        0         0          1         0          0       1     0   
2     0   18        0         0          1         1          1       1     1   
3     0   20        0         0          4         0          1       1     1   
4     1   16        1         1          4         2          0       1     0   
..  ...  ...      ...       ...        ...       ...        ...     ...   ...   
70    0   16        0         1          3         3          1       0     1   
71    1   17        0         1          1         3          1       1     0   
72    0   19        1         1          4         3          0       1     0   
73    1   22        1         1          1         1          0       0     0   
74    1   22        0         0          3         0          0       0     0   

    activities  ...  famrel

In [4]:
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['G3'])
y= train_data['G3']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

In [5]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.42016806722689076


In [6]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in range(200,2000,200)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [7]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=800; total time=   1.2s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1200; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1800; total time=   2.7s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1200; total time=   2.2s
[CV] END bootstrap=True, max_depth=30,

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800]},
                   random_state=42, verbose=2)

In [8]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': False}

In [9]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print (accuracy)
    print(confusion_matrix(y_test,y_pred))
    

best_random = rf_random.best_estimator_
evaluate(best_random, X_test, y_test)

0.46218487394957986
[[11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  1  0  5  2  2  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  1  1  3  1  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  1  1 11  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  5  2  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  3  4  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  3  4  4  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  3  3  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  7  2  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  2  2  1  2  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]]


In [10]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [50,75,100],
    'max_features': ['sqrt'],
    'min_samples_leaf': [4,6,8],
    'min_samples_split': [10,15,20],
    'n_estimators': [1400,1500]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [50, 75, 100],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [4, 6, 8],
                         'min_samples_split': [10, 15, 20],
                         'n_estimators': [1400, 1500]},
             verbose=2)

In [11]:
best_grid = grid_search.best_estimator_
y_pred = best_grid.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred))
print(grid_search.best_params_)
#{'bootstrap': True, 'max_depth': 10, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 1200}

0.4789915966386555
{'bootstrap': False, 'max_depth': 75, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1400}


In [12]:
print(test_data)

    sex  age  address  guardian  studytime  failures  schoolsup  famsup  paid  \
0     0   18        1         0          1         3          1       0     1   
1     0   18        0         0          1         0          0       1     0   
2     0   18        0         0          1         1          1       1     1   
3     0   20        0         0          4         0          1       1     1   
4     1   16        1         1          4         2          0       1     0   
..  ...  ...      ...       ...        ...       ...        ...     ...   ...   
70    0   16        0         1          3         3          1       0     1   
71    1   17        0         1          1         3          1       1     0   
72    0   19        1         1          4         3          0       1     0   
73    1   22        1         1          1         1          0       0     0   
74    1   22        0         0          3         0          0       0     0   

    activities  ...  famrel

In [13]:
sub = best_random.predict(test_data)
output = pd.DataFrame({'Id': list(range(0,75)), 'G3': sub})
output.to_csv('submission.csv', index=False)
print(output.to_string())


    Id  G3
0    0   6
1    1  18
2    2  13
3    3  11
4    4  11
5    5   8
6    6   0
7    7  11
8    8   0
9    9  13
10  10  13
11  11  13
12  12   8
13  13   9
14  14  11
15  15   6
16  16  10
17  17  11
18  18  15
19  19  11
20  20  11
21  21   8
22  22  11
23  23  18
24  24   6
25  25  13
26  26   0
27  27  10
28  28  13
29  29   9
30  30  13
31  31  18
32  32  18
33  33   0
34  34  13
35  35  13
36  36   9
37  37  13
38  38  13
39  39   9
40  40   9
41  41  13
42  42  13
43  43  13
44  44   0
45  45  11
46  46   9
47  47  13
48  48   8
49  49   8
50  50  13
51  51  10
52  52   9
53  53  13
54  54  15
55  55  13
56  56  13
57  57   8
58  58  18
59  59   6
60  60  18
61  61  11
62  62   8
63  63   0
64  64  15
65  65   8
66  66  13
67  67  11
68  68   9
69  69  10
70  70  13
71  71  11
72  72   9
73  73   8
74  74   0
